In [1]:
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

2023-05-31 15:14:33.156327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 15:14:36.741209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# in/out variable lists
vars_mli0 = ['state_t','state_q0001','state_q0002','state_q0003','state_ps']
vars_mlo0 = ['state_t','state_q0001','state_q0002','state_q0003']

In [3]:
# validation dataset for HPO
stride_sample = 5 
f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc')
f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-1[012]-*-*.nc')
f_mli3 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0009-01-*-*.nc')
f_mli_val = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli = f_mli_val
print(f'#files: {len(f_mli_val)}')

#files: 26280


In [4]:
# For testing <faster>
stride_sample = 5 
#f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc')
f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-1[012]-*-*.nc')
#f_mli3 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0009-01-*-*.nc')
#f_mli_val = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli = f_mli2
print(f'#files: {len(f_mli_val)}')

#files: 26280


In [5]:
# new dataset generator function
# that has new options (latlim, lonlim)
mli_mean = xr.open_dataset('./norm_factors/mli_mean.nc')
mli_min = xr.open_dataset('./norm_factors/mli_min.nc')
mli_max = xr.open_dataset('./norm_factors/mli_max.nc')
mlo_scale = xr.open_dataset('./norm_factors/mlo_scale.nc')
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')
def load_nc_mli(filelist:list):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli0]

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #print(ds.shape)
            #dso = dso.stack({'batch':{'sample','ncol'}})
            
            #print(ds.shape, dso.shape)
            yield (ds.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64),
        output_shapes=((None,241))
    )

In [6]:
# new dataset generator function
# that has new options (latlim, lonlim)
mli_mean = xr.open_dataset('./norm_factors/mli_mean.nc')
mli_min = xr.open_dataset('./norm_factors/mli_min.nc')
mli_max = xr.open_dataset('./norm_factors/mli_max.nc')
mlo_scale = xr.open_dataset('./norm_factors/mlo_scale.nc')
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')
def load_nc_mlo(filelist:list):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            ds = ds[vars_mli0]

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            #print(ds.shape)
            #dso = dso.stack({'batch':{'sample','ncol'}})
            
            #print(ds.shape, dso.shape)
            yield (ds.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64),
        output_shapes=((None,240))
    )

In [7]:
tds_testi  = load_nc_mli(f_mli)
worki = list(tds_testi.as_numpy_iterator())
x_true = np.concatenate([ worki[k] for k in range(len(worki)) ])
tds_testo  = load_nc_mlo(f_mli)
worko = list(tds_testi.as_numpy_iterator())
y_true = np.concatenate([ worko[k] for k in range(len(worko)) ])

2023-05-31 15:15:43.509495: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-31 15:15:43.682336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [8]:
T_in = x_true[:,:60]
qv_in = x_true[:,60:120]
ql_in = x_true[:,120:180]
qs_in = x_true[:,180:240]
ps_in = x_true[:,240]

In [9]:
T_out = y_true[:,:60]
qv_out = y_true[:,60:120]
ql_out = y_true[:,120:180]
qs_out = y_true[:,180:240]
ps_out = y_true[:,240]

In [10]:
Tv_in = T_in*(1 + 0.61*qv_in);
Tv_out = T_out*(1 + 0.61*qv_out);

In [11]:
# Convert to pressure
def convert_to_Pressure(hyam,hybm,PS,P0):
    Dimension1=hyam.shape
    Dimension2=PS.shape
    Pressure = np.zeros([Dimension2[0],Dimension1[0]])
    
    for i in range(Dimension2[0]):
        #temp = (P0*hyam[:] + PS[i]*hybm[:])
        #print(temp.shape)
        Pressure[i,:] = (P0*hyam[:] + PS[i]*hybm[:])
    return Pressure

In [12]:
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')
lon = ne4_grid_info.lon.values
lat = ne4_grid_info.lat.values
area = ne4_grid_info.area.values
hyam = ne4_grid_info.hyam.values
hybm = ne4_grid_info.hybm.values
PS = ne4_grid_info.PS.values
P0 = ne4_grid_info.P0.values

In [13]:
Pre_in = convert_to_Pressure(hyam,hybm,ps_in,P0)
Pre_out = convert_to_Pressure(hyam,hybm,ps_out,P0)

In [14]:
ncol = 384
nlev = 60
tlen = 2543616

In [15]:
constRdry  = 287              # J/K/kg
cp_h2o_o   = 1869.4           #evaluated at tk_o and p_o
cp_air_o   = 1005.2           #evaluated at tk_o and p_o
cw_h2o_o   = 4218.0           #evaluated at tk_o
ci_h2o_o   = 2106.0           #evaluated at tk_o
latent_heat_vaporization = 2.501e6  # Latent heat of vaporization in J/kg
rho_in     = Pre_in/(constRdry*Tv_in)
rho_out     = Pre_out/(constRdry*Tv_out)
cpmix      = cp_air_o + cp_h2o_o*qv_in + cw_h2o_o*ql_in+ci_h2o_o*qs_in;
cpmixo      = cp_air_o + cp_h2o_o*qv_out + cw_h2o_o*ql_out+ci_h2o_o*qs_out;

In [16]:
mweight = np.zeros([tlen,nlev])
mweighto = np.zeros([tlen,nlev])
for k in range(nlev):
    mweight[:,k]    =  rho_in[:,k]/np.sum(rho_in,axis=1)
    mweighto[:,k]    = rho_out[:,k]/np.sum(rho_out,axis=1)

In [32]:
# new dataset generator function
# that has new options (latlim, lonlim)

mli_mean = xr.open_dataset('./norm_factors/mli_mean.nc')
mli_min = xr.open_dataset('./norm_factors/mli_min.nc')
mli_max = xr.open_dataset('./norm_factors/mli_max.nc')
mlo_scale = xr.open_dataset('./norm_factors/mlo_scale.nc')
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')

def load_nc_dir_with_generator_test(filelist:list, latlim=[-999,999], lonlim=[-999,999]):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]
            ds = ds.merge(ne4_grid_info[['lat','lon']])
            ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
            ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            dso = dso.merge(ne4_grid_info[['lat','lon']])
            dso = dso.where((dso['lat']>latlim[0])*(dso['lat']<latlim[1]),drop=True)
            dso = dso.where((dso['lon']>lonlim[0])*(dso['lon']<lonlim[1]),drop=True)
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalizatoin, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

In [33]:
# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

In [19]:
tds_test = load_nc_dir_with_generator_test(f_mli)
work = list(tds_test.as_numpy_iterator())
x_true_n = np.concatenate([ work[k][0] for k in range(len(work)) ])
y_true_n = np.concatenate([ work[k][1] for k in range(len(work)) ])

2023-05-31 15:21:36.782021: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [20]:
# This function turns 20 minute increments into daily averages
# (384)*ndays*72 = Dim[0]
# Output dim: (384)*ndays
# Griffin's original version is based on regrided data
# This version is based on ne4pg2 grid (384 columns)
# This change allow me to use Sungduk's code directly
# since he generaged the min/max/scale already
def average_data_Griffin_LiranEdited(reconstructed_targets,reconstructed_features):
    Dim = reconstructed_targets.shape
    x = 384
    tnum = 72
    t = Dim[0]
    z = Dim[1]
    ndays = int(t/(tnum*x))
    
    target_days = np.zeros(shape=(x*ndays,tnum, z))
    feat_days = np.zeros(shape=(x*ndays,tnum, z))
    day_array_targ = np.zeros(shape=(x,tnum,ndays, z))
    day_array_feat = np.zeros(shape=(x,tnum,ndays, z))
    #print(day_array_feat.shape)
    ncol_count = 0
    tstep_count = 0
    day_count = 0
    
    for i in range(t):
        temp_targ = reconstructed_targets[i, :]
        day_array_targ[ncol_count,tstep_count,day_count, :] = temp_targ
        temp_feat = reconstructed_features[i, :]
        day_array_feat[ncol_count,tstep_count,day_count,:] = temp_feat
        
        if (ncol_count == x-1):
            ncol_count = 0
            tstep_count = tstep_count+1
        else:
            ncol_count = ncol_count+1
        
        if (tstep_count == tnum):
            tstep_count = 0
            day_count = day_count+1   
            
            
    day_array_targ_out = np.nanmean(day_array_targ, axis = 1)
    day_array_feat_out = np.nanmean(day_array_feat, axis = 1)
    
    return day_array_targ_out,day_array_feat_out

In [21]:
import os
import netCDF4 as nc
folder_path = "/pscratch/sd/h/heroplr/step2_retrain/backup_phase-4_retrained_models"  # Replace with the actual folder path
out_folder = "/pscratch/sd/h/heroplr/R2_analysis_all/"
Dim_true = x_true.shape
# Loop through all files in the folder
numday = int(Dim_true[0]/ncol/72)
filename="step2_lot-89_trial_0023.best.h5"
file_path = os.path.join(folder_path, filename)

In [22]:
model = keras.models.load_model(file_path,compile=False)
y_pred = model(x_true_n)
T_tend_true = y_true_n[:,:60]
T_pred_true = y_pred[:,:60]
Q_tend_true = y_true_n[:,60:120]
Q_pred_true =y_pred[:,60:120]

In [23]:
T_pre = T_in+T_pred_true*1200
q_pre = qv_in+Q_pred_true*1200
Tv_pre = T_pre*(1 + 0.61*q_pre);
rho_pre     = Pre_in/(constRdry*Tv_pre)
cpmixpre      = cp_air_o + cp_h2o_o*q_pre + cw_h2o_o*ql_in+ci_h2o_o*qs_in;

In [24]:
mweightpre = np.zeros([tlen,nlev])
for k in range(nlev):
    mweightpre[:,k]    =  rho_pre[:,k]/np.sum(rho_pre,axis=1)

In [25]:
# Ideally, we should calculate mweight for pred also, but here I am using an estimated weight for the true. 
T_tend_true_mweight    = T_tend_true*mweighto*cpmixo
T_pred_true_mweight    = T_pred_true*mweightpre*cpmixpre
Q_tend_true_mweight    = Q_tend_true*mweighto*latent_heat_vaporization
Q_pred_true_mweight    = Q_pred_true*mweightpre*latent_heat_vaporization 

In [26]:
T_tend_true2,T_pred_true2 = average_data_Griffin_LiranEdited(T_tend_true_mweight,T_pred_true_mweight)
Q_tend_true2,Q_pred_true2 = average_data_Griffin_LiranEdited(Q_tend_true_mweight,Q_pred_true_mweight)

In [27]:
# Create a new NetCDF file
filename = file_path[-31:]+"_mwv2.nc"
file_path_out = os.path.join(out_folder, filename)
        
ncfile = nc.Dataset(file_path_out, "w", format="NETCDF4")

# Define the dimensions
time_dim = ncfile.createDimension("time", None)  # Unlimited dimension
lat_dim = ncfile.createDimension("ncol", ncol)
lon_dim = ncfile.createDimension("nlev", nlev)
day_dim = ncfile.createDimension("nday", numday)
# Create variables
time_var = ncfile.createVariable("time", "f8", ("time",))
lon_var = ncfile.createVariable("lon", "f4", ("ncol",))
lat_var = ncfile.createVariable("lat", "f4", ("ncol",))
data_var3 = ncfile.createVariable("mwT_tend", "f8", ("nlev","nday","ncol"))
data_var4 = ncfile.createVariable("mwT_pred", "f8", ("nlev","nday","ncol"))
data_var5 = ncfile.createVariable("mwQ_tend", "f8", ("nlev","nday","ncol"))
data_var6 = ncfile.createVariable("mwQ_pred", "f8", ("nlev","nday","ncol"))

# Assign values to variables
time_var[:] = [1]  # Example time values
lon_var[:] = lon    # Example latitude values
lat_var[:] = lat  # Example longitude values

data_var3[:,:,:] = np.transpose(T_tend_true2)
data_var4[:,:,:] = np.transpose(T_pred_true2)
data_var5[:,:,:] = np.transpose(Q_tend_true2)
data_var6[:,:,:] = np.transpose(Q_pred_true2)
# Add global attributes
ncfile.description = "R2"
ncfile.history = "Created by Liran"

# Close the NetCDF file
ncfile.close()
    